In [1]:
import pandas as pd
import os
import basedosdados as bd
import re
import unicodedata
import numpy as np
import glob
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time as tm

In [ ]:
#disclamer -> é importante verificar se as versões do chrome e do webdriver são compatíves.
folder_location = 'br_me_empresas_exportadoras_importadoras/input'
driver_path = 'C:/webdrivers/chromedriver.exe'
folder_donwload = r"br_me_empresas_exportadoras_importadoras\input"

def baixar_tabelas_exportacao_importacao(url: str) -> pd:
    options = Options()
    options.add_experimental_option("prefs", 
    {"download.default_directory": folder_donwload})
    navegador = webdriver.Chrome(executable_path = driver_path, options = options)
    navegador.get(url)
    #atencao ##
    #precisa ser o tempo necessario para carregar a página e baixar a tabela 
    tm.sleep(20)

#inserir o ano corrente para atualizar
for year in range(1997,2022):
    url1 = f'https://balanca.economia.gov.br/balanca/outras/EMPRESAS_CADASTRO_{year}.xlsx' 
    baixar_tabelas_exportacao_importacao(url1)
    

In [2]:
folder_location = 'br_me_empresas_exportadoras_importadoras/input'
filenames = glob.glob(folder_location + "/*.xlsx")

lista_empresas = []

##checar de fato se cada ano pra imp e exp tem so um registro de cnpj
for file in filenames: 
    #ler folhas de um mesmo ano

    #data é a folha com empresas exportadoras
    #data1 é a folha com importadoras
    data = pd.read_excel(file, sheet_name = 0, skiprows = 7, dtype=str)
    data1 = pd.read_excel(file, sheet_name = 1, skiprows = 7, dtype=str)
    
    #dava erro de Length of values (1) does not match length of index (15293)
    #o que nao ocorre se o argumento passado é uma string.
    
    data1['id_exportacao_importacao_tmp'] = '1'
    data['id_exportacao_importacao_tmp'] = '0'

    df_empresas = pd.concat([data1, data])
    
    #nas tabelas originais, não existe a coluna ano
    #optei por adicionar uma coluna com o ano que consta no nome do arquivo no momento da leitura de cada planilha
    ano = re.findall(r'\d+', file)
    df_empresas['ano'] = ano[0]
    
    #criar a variavel id_exportacao_importacao que contara com 3 entradas:
    #0 -> empresas exportadoras no ano de referencia
    #1 -> empresas importadoras no ano de referencia
    #2 -> empresas exportadoras e importadoras no ano de referencia
    df_empresas['id_exportacao_importacao'] = np.where(df_empresas['CNPJ'].duplicated(keep=False),
    '2', df_empresas['id_exportacao_importacao_tmp'])
    #dropar a coluna temporaria ('id_exportacao_importacao_tmp') e dar unique para tirar as repeticoes 
    df_empresas = df_empresas.drop(columns={'id_exportacao_importacao_tmp'})
    df_empresas = df_empresas.drop_duplicates() 
    
    lista_empresas.append(df_empresas)
    
df_empresas = pd.concat(lista_empresas)  

In [ ]:
#baixar diretorio de municipios da bd
df = bd.read_table(dataset_id='br_bd_diretorios_brasil',
table_id='municipio',
billing_project_id="inserir-projeto")

df_cnae2 = bd.read_table(dataset_id='br_bd_diretorios_brasil',
table_id='cnae_2',
billing_project_id="inserir-projeto")

In [4]:
#renomear variaveis
df_empresas = df_empresas.rename(columns = {'CNPJ': 'cnpj',
    'EMPRESA' : 'razao_social',
    'ENDEREÇO' : 'endereco',
    'NÚMERO' : 'numero',
    'BAIRRO' : 'bairro',
    'CEP' : 'cep',
    'MUNICÍPIO' : 'nome',
    'UF' : 'sigla_uf',
    'CNAE PRIMÁRIA' : 'cnae_2_primaria',
    'NATUREZA JURÍDICA' : 'natureza_juridica',
    'ano' : 'ano',
    'id_exportacao_importacao' : 'id_exportacao_importacao'
    })

#fazer alteracoes manuais nos nomes de municipios preenchidos errados na base
#transformar ambos nomes de municipios em caixa baixa e retirar acentos
df['nome'] = df['nome'].str.lower()
df['nome'] = df['nome'].str.strip()
df_empresas['nome'] = df_empresas['nome'].str.lower()
df_empresas['nome'] = df_empresas['nome'].str.strip()

#Remover acentos
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

df['nome'] = df['nome'].apply(remove_accents)


df_empresas['nome'] = np.where(df_empresas['nome'] == 'embu', 'embu das artes' , df_empresas['nome'])
#apesar de existirem 2 municipios que iniciam com o nome 'embu' 
#o municipio de 'embu-guaçu' possui somente um cep 	06900-000.
#nenhuma empresa tem esse cep registrado, logo todas estao em embu das artes
df_empresas['nome'] = np.where(df_empresas['nome'] == 'belem de sao francisco' ,'belem do sao francisco' , df_empresas['nome'])
df_empresas['nome'] = np.where(df_empresas['nome'] == 'mogi mirin', 'mogi mirim' , df_empresas['nome'])
df_empresas['nome'] = np.where(df_empresas['nome'] == 'santana do livramento' ,  "sant'ana do livramento" , df_empresas['nome'])
df_empresas['nome'] = np.where(df_empresas['nome'] == 'brasopolis',  'brazopolis' , df_empresas['nome'])
df_empresas['nome'] = np.where(df_empresas['nome'] == 'picarras' ,  'balneario picarras' , df_empresas['nome'])
df_empresas['nome'] = np.where(df_empresas['nome'] == 'lagoa do itaenga' ,'lagoa de itaenga' , df_empresas['nome'])
df_empresas['nome'] = np.where(df_empresas['nome'] == 'couto de magalhaes' ,   'couto magalhaes' , df_empresas['nome'])
df_empresas['nome'] = np.where(df_empresas['nome'] == 'parati', 'paraty' , df_empresas['nome'])
df_empresas['nome'] = np.where(df_empresas['nome'] == 'eldorado dos carajas' ,  'eldorado do carajas' , df_empresas['nome'])
df_empresas['nome'] = np.where(df_empresas['nome']== 'sao valerio da natividade' , 'sao valerio' , df_empresas['nome'])
df_empresas['nome'] = np.where(df_empresas['nome'] == 'presidente juscelino'  , 'serra caiada' , df_empresas['nome'])      #erro de prenchimento no cadastro  
df_empresas['nome'] = np.where(df_empresas['nome'] == 'serido'  ,'sao vicente do serido' , df_empresas['nome']) 
df_empresas['nome'] = np.where(df_empresas['nome'] == 'mogi-mirim'  ,'mogi mirim' , df_empresas['nome']) 

In [6]:
#fazer o merge com os municipios para pegar o id_municipio
df_empresas = pd.merge(df_empresas,
df[['nome', 'sigla_uf', 'id_municipio']],
on = ['nome', 'sigla_uf'],
how = 'left')



In [5]:
#splitar as variaveis que não foram normalizadas na base original
df_empresas[['id_natureza_juridica', 'natureza_juridica_descricao']] = df_empresas.natureza_juridica.str.split("-", expand=True, n = 1)
df_empresas[['cnae_2_primaria', 'cnae_2_primaria_descricao']] = df_empresas.cnae_2_primaria.str.split("-", expand=True, n = 1)
df_empresas['cnae_2_primaria'] = df_empresas['cnae_2_primaria'].str.replace(' ', '')


In [7]:
#tirar pontos e numeros da coluna cnae_2 do diretorio da BD.
#retirar os 4 primeiros digitos e dar merge 

df_cnae2['cnae_2_tmp'] = df_cnae2['cnae_2'].str.replace('-','')
df_cnae2['cnae_2_tmp'] = df_cnae2['cnae_2_tmp'].str.replace('.','')
df_cnae2['cnae_2_tmp'] = df_cnae2['cnae_2_tmp'].str.slice(0, 4)

df_empresas = pd.merge(df_empresas,
df_cnae2[['cnae_2', 'cnae_2_tmp', 'descricao']],
left_on = 'cnae_2_primaria',
right_on = 'cnae_2_tmp',
how = 'left')



In [11]:
df_empresas = df_empresas.drop(columns=['natureza_juridica_descricao', 'cnae_2_primaria_descricao',
'cnae_2_primaria', 'descricao', 'cnae_2_tmp'])

df_empresas = df_empresas.rename(columns = {'cnae_2' : 'cnae_2_primaria'})

#ordernar colunas de acordo com a tabela de arquitetura
df_empresas = df_empresas.loc[:, ["ano","cnpj","razao_social","id_exportacao_importacao","cnae_2_primaria", "id_natureza_juridica", "sigla_uf", "id_municipio", "cep", "bairro","endereco", "numero"]]



In [14]:
#salvar df final particionado por ano
anos = df_empresas['ano'].unique()
for ano in anos:
    #criar diretorio com ano
    os.mkdir(f'br_me_empresas_exportadoras_importadoras/output/ano={ano}')
    #filtrar ano, dropar coluna ano e salvar
    df_empresas.query(f"ano == {ano}").drop(columns =['ano']).to_csv(f'C:/Users/gabri/OneDrive/vida_profissional/Projetos/base_dos_dados/br_me_empresas_exportadoras_importadoras/output/ano={ano}/estabelecimentos.csv', 
    sep = ',', index = False, na_rep= ' ', encoding = 'utf-8')

In [2]:
#criar dicionairo com id_exportacao_importacao
#atualizar cobertura temporal em atualizacoes da base
dados =  { 'id_tabela' : 'empresas_exportadoras_importadoras', 'nome_coluna' : 'id_exportacao_importacao',  
'chave': ['0', '1', '2'], 'cobertura_temporal' : '1997(1)2021' , 'valor' : ['exportou', 'importou', 'exportou e importou'] }

dicionario = pd.DataFrame(data=dados)
#salvar dic
dicionario.to_csv('br_me_empresas_exportadoras_importadoras/extra/dicionario.csv', 
sep = ',', index = False, na_rep= ' ', encoding = 'utf-8')